<h4>Притула М.І. 124М-22-1 Лаб№1 Варіант 4</h4>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import scipy.stats as stats
from scipy.stats import chisquare
from scipy.linalg import qr, svd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_excel("lab1var4.xlsx")
df = df.drop('i', axis = 1)
# Замена запятых на точки
df = df.apply(lambda x: x.str.replace(',', '.'))
# Преобразование значений в тип float
df = df.astype(float)
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,Y
0,65.008749,6481.219350,0.661075,2.876553,64.782726,921.818160,1174.816715,3.774410,17.352574,84.052033,163.932347
1,70.276340,3071.648209,0.762387,5.048826,134.787671,1021.562880,1180.805518,3.946171,19.222675,102.553159,194.585281
2,77.227788,2789.703091,0.452850,5.984970,126.028495,999.262708,1168.262414,5.435011,19.278947,99.965332,209.946115
3,80.132466,4906.129908,0.233224,5.455785,182.390696,933.410705,1136.308313,5.072538,18.062633,80.313876,204.042521
4,85.503007,2467.673775,0.289270,3.590833,192.529910,995.566549,1106.339439,5.518110,19.232582,81.108089,199.710298


In [4]:
df.corr().style.background_gradient(cmap='coolwarm', axis = 1)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,Y
x1,1.000000,0.033419,0.079243,-0.371507,0.907835,0.247541,-0.178248,0.997541,0.222415,0.528870,0.694504
x2,0.033419,1.000000,0.004605,0.036034,-0.109759,-0.070164,-0.067236,0.045612,-0.132060,-0.080170,0.044640
x3,0.079243,0.004605,1.000000,0.173399,0.033300,-0.106578,0.482017,0.073551,-0.154966,0.086659,0.194391
x4,-0.371507,0.036034,0.173399,1.000000,-0.386526,-0.277991,0.112915,-0.394794,-0.259542,-0.309121,0.401340
x5,0.907835,-0.109759,0.033300,-0.386526,1.000000,0.195778,-0.227105,0.900358,0.204244,0.467902,0.591877
x6,0.247541,-0.070164,-0.106578,-0.277991,0.195778,1.000000,0.128877,0.249118,0.981138,0.526377,0.136747
x7,-0.178248,-0.067236,0.482017,0.112915,-0.227105,0.128877,1.000000,-0.168213,0.114177,0.097049,-0.068361
x8,0.997541,0.045612,0.073551,-0.394794,0.900358,0.249118,-0.168213,1.000000,0.220693,0.528518,0.673543
x9,0.222415,-0.132060,-0.154966,-0.259542,0.204244,0.981138,0.114177,0.220693,1.000000,0.548291,0.129444
x10,0.528870,-0.080170,0.086659,-0.309121,0.467902,0.526377,0.097049,0.528518,0.548291,1.000000,0.330310


В отриманій таблиці бачимо коллінеарність між: x1 x5, x5 x8, x6 x9 , x8 x1

In [5]:
scaled_X = MinMaxScaler().fit_transform(df.drop('Y', axis = 1))
df_scaled = pd.DataFrame(scaled_X, columns = df.drop('Y', axis = 1).columns)
df_scaled['Y'] = df['Y']
df_scaled.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,Y
0,0.000000,0.985008,0.657969,0.740639,0.000000,0.089211,0.763560,0.000000,0.066420,0.410064,163.932347
1,0.038205,0.253743,0.771672,0.921890,0.236304,0.483158,0.826696,0.010970,0.460886,0.734999,194.585281
2,0.088623,0.193273,0.424278,1.000000,0.206737,0.395082,0.694461,0.106061,0.472756,0.689549,209.946115
3,0.109690,0.647192,0.177791,0.955846,0.396989,0.134996,0.357588,0.082910,0.216195,0.344411,204.042521
4,0.148641,0.124206,0.240692,0.800237,0.431214,0.380484,0.041644,0.111368,0.462976,0.358360,199.710298


In [6]:
def farrar_glauber(df):
    # Separate dependent and independent variables
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    
    # Calculate residual errors for each independent variable
    residuals = []
    for i in range(X.shape[1]):
        # Fit linear regression model on independent variable i
        X_temp = X.drop(columns=X.columns[i])
        model = np.linalg.lstsq(X_temp, X.iloc[:,i], rcond=None)
        y_pred = np.dot(X_temp, model[0])
        
        # Calculate residual errors and append to list
        residuals.append(X.iloc[:,i] - y_pred)
    
    # Calculate correlation matrix of residuals
    corr_matrix = np.corrcoef(residuals)
    n = corr_matrix.shape[0]
    
    # Calculate T values and p-values
    T_values = np.zeros((n,n))
    p_values = np.zeros((n,n))
    for i in range(n):
        for j in range(i+1, n):
            r = corr_matrix[i,j]
            T_values[i,j] = r * np.sqrt((n-2)/(1-r**2))
            p_values[i,j] = 2*(1 - stats.t.cdf(abs(T_values[i,j]), n-2))
    
    # Create DataFrame to store T values and p-values
    col_names = X.columns
    t_values_df = pd.DataFrame(T_values, columns=col_names, index=col_names)
    p_values_df = pd.DataFrame(p_values, columns=col_names, index=col_names)
    
    # Return results as tuple
    return t_values_df, p_values_df

In [7]:
t_values, p_values = farrar_glauber(df_scaled)

In [8]:
pd.set_option('display.float_format', lambda x: '%.12f' % x)

In [9]:
alpha = 0.05
p_values

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
x1,0.000000000000,0.557336383825,0.661192089854,0.391896146492,0.700602474770,0.782668606564,0.442711100606,0.000000109002,0.752375952451,0.834889784900
x2,0.000000000000,0.000000000000,0.986921413123,0.257566700166,0.944917825623,0.558083110880,0.925185604191,0.542262097509,0.571761773347,0.585590210960
x3,0.000000000000,0.000000000000,0.000000000000,0.548072036274,0.648074668206,0.410060912654,0.098053912239,0.642711412758,0.334695243168,0.538582437486
x4,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.749357178578,0.662391153636,0.676169146032,0.340793759024,0.685629184049,0.806055542067
x5,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.265256341993,0.891329915067,0.995804998063,0.253179547863,0.791311282901
x6,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.871727411168,0.648463635030,0.000000506666,0.424762547415
x7,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.463329645115,0.771530183728,0.727626962698
x8,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.617430751250,0.762237099815
x9,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.265056594970
x10,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000


Після проведення тесту Фаррара-Глобера pvalues x9->x6, x8->x1 менше рівня значимості 0.05, тому ці признаки дійсно мультиколінеарні, та одні з них варто видалити для ліпшої якості моделі

In [10]:
X_scaled = df_scaled.drop(['x9', 'x8'], axis = 1)

In [11]:
X = X_scaled.drop('Y', axis=1)

In [12]:
X_scaled['Y'] = df['Y']
X_scaled

,x1,x2,x3,x4,x5,x6,x7,x10,Y
0,0.000000000000,0.985007708649,0.657969179376,0.740639154787,0.000000000000,0.089211090363,0.763559561936,0.410063989105,163.932347000000
1,0.038204928976,0.253742592271,0.771672206634,0.921889783030,0.236303698906,0.483157906633,0.826696020832,0.734998791636,194.585280500000
2,0.088622581785,0.193272632560,0.424277718024,1.000000000000,0.206736850603,0.395082249260,0.694461387280,0.689548860438,209.946115200000
3,0.109689712235,0.647191626661,0.177791037577,0.955845700316,0.396989073901,0.134996433046,0.357587927870,0.344410827021,204.042520900000
4,0.148641323902,0.124205645686,0.240691831855,0.800237396570,0.431214281041,0.380484083964,0.041643559446,0.358359567260,199.710297800000
5,0.191121456278,0.386131499951,0.264020223049,0.571273428411,0.307714144649,0.789455213742,0.647991382797,0.000000000000,189.682763200000
6,0.244329440891,0.640295133429,0.363473407618,0.325012160818,0.499519944550,0.335210965208,0.334060932303,0.705422098962,167.791177500000
7,0.284794513063,0.627223511707,0.123066500206,0.121746940602,0.326392357400,0.911824450989,0.266426234898,0.839559239594,162.318357900000
8,0.300895787328,0.766018004933,0.031628616466,0.011244182891,0.482588377502,0.252441681870,0.109167993711,0.507316109257,142.883750900000
9,0.331660289390,0.354125871967,0.704200511631,0.020558816643,0.487159960949,0.017771645370,0.970766720895,0.582350342464,145.643469000000


Add-Del алгоритм

In [14]:
def add_delete_algorithm(train_data, test_data, max_factors):
    """
    Add-Delete algorithm for selecting informative features.

    Args:
        train_data (pd.DataFrame): Training data with features and target.
        test_data (pd.DataFrame): Test data with features and target.
        max_factors (int): Maximum number of features to consider.

    Returns:
        (list): List of selected features.
    """
    # Start with an empty set of features.
    features = set()

    # Step 2: Identify dependencies on training data and compute errors on test data.
    best_error = float('inf')
    best_feature = None
    for feature in train_data.drop('Y', axis=1).columns:
        # Add feature to set.
        features.add(feature)

        # Train model on training data.
        model = LinearRegression().fit(train_data[list(features)], train_data['Y'])

        # Evaluate model on test data.
        test_predictions = model.predict(test_data[list(features)])
        test_error = mean_squared_error(test_data['Y'], test_predictions)

        # Update best feature if necessary.
        if test_error < best_error:
            best_error = test_error
            best_feature = feature

        # Remove feature from set.
        features.remove(feature)

    # Step 3: Add best feature to set.
    features.add(best_feature)

    # Step 4: Repeat until maximum number of features is reached.
    while len(features) < max_factors:
        # Step 5: Identify dependencies on training data and compute errors on test data.
        best_error = float('inf')
        best_feature = None
        for feature in train_data.drop('Y', axis=1).columns:
            if feature in features:
                continue

            # Add feature to set.
            features.add(feature)

            # Train model on training data.
            model = LinearRegression().fit(train_data[list(features)], train_data['Y'])

            # Evaluate model on test data.
            test_predictions = model.predict(test_data[list(features)])
            test_error = mean_squared_error(test_data['Y'], test_predictions)

            # Update best feature if necessary.
            if test_error < best_error:
                best_error = test_error
                best_feature = feature

            # Remove feature from set.
            features.remove(feature)

        # Step 6: Add best feature to set.
        features.add(best_feature)

    # Step 7: Return list of selected features.
    return list(features)

In [16]:
# Split data into training and test sets.
train_data, test_data = train_test_split(X_scaled, test_size=0.2, random_state=42)

# Run Add-Delete algorithm.
selected_features = add_delete_algorithm(train_data, test_data, max_factors=5)

# Train final model on selected features.
final_model = LinearRegression().fit(train_data[selected_features], train_data['Y'])

# Evaluate final model on test data.
test_predictions = final_model.predict(test_data[selected_features])
test_error = mean_squared_error(test_data['Y'], test_predictions)
print(f'Test error: {test_error:.2f}')

Test error: 0.80


In [17]:
selected_features

['x2', 'x6', 'x4', 'x1', 'x10']

Del-Add алгоритм

In [18]:
def del_add_algorithm(x_train, y_train, x_test, y_test, max_factors):
    n_features = x_train.shape[1]
    selected_factors = set(range(n_features))
    curr_factors = selected_factors.copy()
    curr_score = np.inf
    
    while len(selected_factors) <= max_factors:
        # Try deleting each factor from the current set
        for factor in curr_factors:
            new_factors = curr_factors.copy()
            new_factors.remove(factor)
            model = LinearRegression().fit(x_train[:, list(new_factors)], y_train)
            score = mean_squared_error(y_test, model.predict(x_test[:, list(new_factors)]))
            
            # If the score improved, update the current set of factors and score
            if score < curr_score:
                curr_factors = new_factors
                curr_score = score
        
        # Try adding each factor that isn't in the current set
        for factor in selected_factors - curr_factors:
            new_factors = curr_factors.copy()
            new_factors.add(factor)
            model = LinearRegression().fit(x_train[:, list(new_factors)], y_train)
            score = mean_squared_error(y_test, model.predict(x_test[:, list(new_factors)]))
            
            # If the score improved, update the current set of factors and score
            if score < curr_score:
                curr_factors = new_factors
                curr_score = score
        
        # If no improvements were made, stop the algorithm
        if curr_factors == selected_factors:
            break
        
        # Otherwise, update the selected set of factors and continue
        selected_factors = curr_factors.copy()
    
    return selected_factors

In [19]:
X_train = train_data.drop('Y', axis = 1)
y_train = train_data['Y']
X_test = test_data.drop('Y', axis = 1)
y_test = test_data['Y']
selected_features = del_add_algorithm(X_train, y_train, X_test, y_test, 5)

In [20]:
list(selected_features)

[0, 1, 2, 3, 4, 5, 6, 7]

In [21]:
# Train final model on selected features.
final_model = LinearRegression().fit(train_data[X_train.columns[list(selected_features)]], train_data['Y'])

# Evaluate final model on test data.
test_predictions = final_model.predict(test_data.drop('Y', axis = 1))
test_error = mean_squared_error(test_data['Y'], test_predictions)
print(f'Test error: {test_error:.2f}')

Test error: 1.16


**Висновок**:<br>
Тест Фаррара-Глобера демонструє колінеарність між двома незалежними змінними.<br>
Add-Del алгоритм повернув фічі: ['x2', 'x6', 'x4', 'x1', 'x10'].<br> Del-Add алгоритм повернув фічі: [0, 1, 2, 3, 4, 5, 6, 7]